In [31]:
import os
os.getcwd()
from infrastructure import *
import datasets

In [2]:
# test
model_wrapped = EncoderWrapper(device="cuda")
test_dataset_xor = datasets.load_dataset("data_scripts/data_xor.py", add_sep=False)["train"]
# test_dataloader = DataLoader(test_dataset_xor, batch_size=3)

# print(next(iter(test_dataloader)))
# output = model_wrapped(next(iter(test_dataloader))["content"])
# print(len(output))
# print(output[0].size())


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Found cached dataset data_xor (/home2/shashwat.s/main_thread/negation_new/{'cache_dir': None, 'config_name': None, 'data_dir': None, 'data_files': None, 'hash': 'd2a449d4e373869e0295a5e1aff1c8052abb878b8e40f9c2241d9639b4a4bf72', 'features': None, 'use_auth_token': None, 'base_path': 'data_scripts

  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
ic.disable()
outs = get_hidden_states_many_examples(model_wrapped, test_dataset_xor, n=4, batch_size=2)
print(len(outs[0]))
print(outs[0][0].size())

13
torch.Size([4, 768])



# The XOR experiment

We have negated and non-negated versions of the same propositions in LAMA

- A: Einstein was born in Austria
- A': Eisntein was not born in Austria

We make the following combinations
- AA
- AA'
- A'A
- A'A'

We classify contradictory statements together and non-contradictory statements together

The point is to ask if a linear regression can seperate out this classification, the point is to look if the model is doing compositionality.


In [6]:
ic.disable()
model_wrapped = EncoderWrapper(device="cuda")
train_dataset_xor = datasets.load_dataset("data_scripts/data_xor.py")["train"]
test_dataset_xor = datasets.load_dataset("data_scripts/data_xor.py")["test"]
dev_dataset_xor = datasets.load_dataset("data_scripts/data_xor.py")["validation"]


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Found cached dataset data_xor (/home2/shashwat.s/main_thread/negation_new/{'cache_dir': None, 'config_name': None, 'data_dir': None, 'data_files': None, 'hash': 'd2a449d4e373869e0295a5e1aff1c8052abb878b8e40f9c2241d9639b4a4bf72', 'features': None, 'use_auth_token': None, 'base_path': 'data_scripts

  0%|          | 0/3 [00:00<?, ?it/s]

Found cached dataset data_xor (/home2/shashwat.s/main_thread/negation_new/{'cache_dir': None, 'config_name': None, 'data_dir': None, 'data_files': None, 'hash': 'd2a449d4e373869e0295a5e1aff1c8052abb878b8e40f9c2241d9639b4a4bf72', 'features': None, 'use_auth_token': None, 'base_path': 'data_scripts'}/data_xor/default/0.0.0)


  0%|          | 0/3 [00:00<?, ?it/s]

Found cached dataset data_xor (/home2/shashwat.s/main_thread/negation_new/{'cache_dir': None, 'config_name': None, 'data_dir': None, 'data_files': None, 'hash': 'd2a449d4e373869e0295a5e1aff1c8052abb878b8e40f9c2241d9639b4a4bf72', 'features': None, 'use_auth_token': None, 'base_path': 'data_scripts'}/data_xor/default/0.0.0)


  0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
train, train_labels = get_hidden_states_many_examples(model_wrapped, train_dataset_xor, n=248, batch_size=100)


In [8]:
test, test_labels = get_hidden_states_many_examples(model_wrapped, test_dataset_xor, n=1000, batch_size=100)


In [9]:
ic(len(test_labels))
ic(test_labels.size())
ic(train_labels.size())

torch.Size([200])

In [10]:
models, accuracies = run_experiment_across_layers(linear_probe_experiment, train, train_labels, test, test_labels)

In [11]:
reports_xor_no_control = generate_classification_report_all_layers(test, test_labels, models)

In [12]:
for report in reports_xor_no_control[:3]:
    print(report)

              precision    recall  f1-score   support

           0       0.52      0.58      0.55       500
           1       0.53      0.47      0.50       500

    accuracy                           0.53      1000
   macro avg       0.53      0.52      0.52      1000
weighted avg       0.53      0.53      0.52      1000

              precision    recall  f1-score   support

           0       0.55      0.58      0.56       500
           1       0.55      0.53      0.54       500

    accuracy                           0.55      1000
   macro avg       0.55      0.55      0.55      1000
weighted avg       0.55      0.55      0.55      1000

              precision    recall  f1-score   support

           0       0.53      0.54      0.54       500
           1       0.54      0.53      0.53       500

    accuracy                           0.54      1000
   macro avg       0.54      0.54      0.53      1000
weighted avg       0.54      0.54      0.53      1000



In [13]:
for report in reports_xor_no_control[3:6]:
    print(report)

              precision    recall  f1-score   support

           0       0.66      0.67      0.66       500
           1       0.66      0.65      0.66       500

    accuracy                           0.66      1000
   macro avg       0.66      0.66      0.66      1000
weighted avg       0.66      0.66      0.66      1000

              precision    recall  f1-score   support

           0       0.78      0.84      0.81       500
           1       0.83      0.77      0.80       500

    accuracy                           0.80      1000
   macro avg       0.81      0.80      0.80      1000
weighted avg       0.81      0.80      0.80      1000

              precision    recall  f1-score   support

           0       0.85      0.83      0.84       500
           1       0.83      0.85      0.84       500

    accuracy                           0.84      1000
   macro avg       0.84      0.84      0.84      1000
weighted avg       0.84      0.84      0.84      1000



In [14]:
for report in reports_xor_no_control[6:9]:
    print(report)

              precision    recall  f1-score   support

           0       0.88      0.90      0.89       500
           1       0.90      0.87      0.88       500

    accuracy                           0.89      1000
   macro avg       0.89      0.89      0.89      1000
weighted avg       0.89      0.89      0.89      1000

              precision    recall  f1-score   support

           0       0.89      0.95      0.92       500
           1       0.94      0.88      0.91       500

    accuracy                           0.92      1000
   macro avg       0.92      0.92      0.91      1000
weighted avg       0.92      0.92      0.91      1000

              precision    recall  f1-score   support

           0       0.91      0.94      0.93       500
           1       0.94      0.91      0.92       500

    accuracy                           0.93      1000
   macro avg       0.93      0.93      0.93      1000
weighted avg       0.93      0.93      0.93      1000



In [15]:
for report in reports_xor_no_control[-3:]:
    print(report)

              precision    recall  f1-score   support

           0       0.92      0.93      0.93       500
           1       0.93      0.92      0.92       500

    accuracy                           0.93      1000
   macro avg       0.93      0.93      0.92      1000
weighted avg       0.93      0.93      0.92      1000

              precision    recall  f1-score   support

           0       0.92      0.93      0.92       500
           1       0.93      0.91      0.92       500

    accuracy                           0.92      1000
   macro avg       0.92      0.92      0.92      1000
weighted avg       0.92      0.92      0.92      1000

              precision    recall  f1-score   support

           0       0.92      0.86      0.89       500
           1       0.87      0.93      0.90       500

    accuracy                           0.89      1000
   macro avg       0.89      0.89      0.89      1000
weighted avg       0.89      0.89      0.89      1000



## Control Experiment
replace negation with gibberish and run the linear probes for inference

In [16]:

test_dataset_xor_1 = datasets.load_dataset("data_scripts/data_xor.py", task_num=1, cache_dir="control_task")["test"]

Found cached dataset data_xor (/home2/shashwat.s/main_thread/negation_new/{'cache_dir': 'control_task', 'config_name': None, 'data_dir': None, 'data_files': None, 'hash': 'd2a449d4e373869e0295a5e1aff1c8052abb878b8e40f9c2241d9639b4a4bf72', 'features': None, 'use_auth_token': None, 'base_path': 'data_scripts', 'task_num': 1}/data_xor/default/0.0.0)


  0%|          | 0/3 [00:00<?, ?it/s]

In [17]:
test_control1, test_labels_control1 = get_hidden_states_many_examples(model_wrapped, test_dataset_xor_1, n=10000, batch_size=100)


In [39]:
test_labels_control1

tensor([0, 1, 1,  ..., 1, 1, 0], device='cuda:0')

In [18]:
reports_xor_control_1 = generate_classification_report_all_layers(test_control1, test_labels_control1, models)

In [19]:
for report in reports_xor_control_1[:3]:
    print(report)

              precision    recall  f1-score   support

           0       0.50      0.96      0.66      3350
           1       0.50      0.04      0.08      3350

    accuracy                           0.50      6700
   macro avg       0.50      0.50      0.37      6700
weighted avg       0.50      0.50      0.37      6700

              precision    recall  f1-score   support

           0       0.51      0.70      0.59      3350
           1       0.53      0.33      0.41      3350

    accuracy                           0.52      6700
   macro avg       0.52      0.52      0.50      6700
weighted avg       0.52      0.52      0.50      6700

              precision    recall  f1-score   support

           0       0.55      0.57      0.56      3350
           1       0.55      0.53      0.54      3350

    accuracy                           0.55      6700
   macro avg       0.55      0.55      0.55      6700
weighted avg       0.55      0.55      0.55      6700



In [20]:
for report in reports_xor_control_1[3:6]:
    print(report)

              precision    recall  f1-score   support

           0       0.71      0.78      0.75      3350
           1       0.76      0.69      0.72      3350

    accuracy                           0.74      6700
   macro avg       0.74      0.74      0.74      6700
weighted avg       0.74      0.74      0.74      6700

              precision    recall  f1-score   support

           0       0.77      0.81      0.79      3350
           1       0.80      0.76      0.78      3350

    accuracy                           0.78      6700
   macro avg       0.79      0.78      0.78      6700
weighted avg       0.79      0.78      0.78      6700

              precision    recall  f1-score   support

           0       0.74      0.89      0.81      3350
           1       0.86      0.69      0.76      3350

    accuracy                           0.79      6700
   macro avg       0.80      0.79      0.79      6700
weighted avg       0.80      0.79      0.79      6700



In [21]:
for report in reports_xor_control_1[-3:]:
    print(report)

              precision    recall  f1-score   support

           0       0.67      0.96      0.79      3350
           1       0.93      0.53      0.67      3350

    accuracy                           0.74      6700
   macro avg       0.80      0.74      0.73      6700
weighted avg       0.80      0.74      0.73      6700

              precision    recall  f1-score   support

           0       0.73      0.96      0.83      3350
           1       0.94      0.65      0.77      3350

    accuracy                           0.80      6700
   macro avg       0.84      0.80      0.80      6700
weighted avg       0.84      0.80      0.80      6700

              precision    recall  f1-score   support

           0       0.64      0.89      0.75      3350
           1       0.82      0.50      0.62      3350

    accuracy                           0.70      6700
   macro avg       0.73      0.70      0.68      6700
weighted avg       0.73      0.70      0.68      6700



In [47]:
import datasets
import os
os.statvfs(datasets.config.HF_DATASETS_CACHE)

os.statvfs_result(f_bsize=1048576, f_frsize=1048576, f_blocks=34191575, f_bfree=9798649, f_bavail=8082217, f_files=549253120, f_ffree=360303603, f_favail=360303603, f_flag=4096, f_namemax=255)

# Negation Detection

Very dumb experiment, checking for negation in the statement (from mean-pooled reps)

In [22]:
train_dataset_neg_detect = datasets.load_dataset("data_scripts/data_detect_neg.py", add_sep=False)["train"]
test_dataset_neg_detect = datasets.load_dataset("data_scripts/data_detect_neg.py", add_sep=False)["test"]
dev_dataset_neg_detect = datasets.load_dataset("data_scripts/data_detect_neg.py", add_sep=False)["validation"]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset data_detect_neg downloaded and prepared to /home2/shashwat.s/main_thread/negation_new/{'cache_dir': None, 'config_name': None, 'data_dir': None, 'data_files': None, 'hash': '1b53f52a30ee637c38ac67e920770d58d6c1da4f3bed41fbe8442d79b6881e56', 'features': None, 'use_auth_token': None, 'base_path': 'data_scripts', 'add_sep': False}/data_detect_neg/default/0.0.0. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Found cached dataset data_detect_neg (/home2/shashwat.s/main_thread/negation_new/{'cache_dir': None, 'config_name': None, 'data_dir': None, 'data_files': None, 'hash': '1b53f52a30ee637c38ac67e920770d58d6c1da4f3bed41fbe8442d79b6881e56', 'features': None, 'use_auth_token': None, 'base_path': 'data_scripts', 'add_sep': False}/data_detect_neg/default/0.0.0)


  0%|          | 0/3 [00:00<?, ?it/s]

Found cached dataset data_detect_neg (/home2/shashwat.s/main_thread/negation_new/{'cache_dir': None, 'config_name': None, 'data_dir': None, 'data_files': None, 'hash': '1b53f52a30ee637c38ac67e920770d58d6c1da4f3bed41fbe8442d79b6881e56', 'features': None, 'use_auth_token': None, 'base_path': 'data_scripts', 'add_sep': False}/data_detect_neg/default/0.0.0)


  0%|          | 0/3 [00:00<?, ?it/s]

In [24]:
train_detect_neg, train_labels_det_neg = get_hidden_states_many_examples(model_wrapped, train_dataset_neg_detect, n=500, batch_size=100)


In [25]:
test_detect_neg, test_labels_det_neg = get_hidden_states_many_examples(model_wrapped, test_dataset_neg_detect, n=2000, batch_size=100)


In [26]:
run_experiment_across_layers(linear_probe_experiment, train_detect_neg, train_labels_det_neg, test_detect_neg, test_labels_det_neg)


([LogisticRegression(class_weight='balanced', max_iter=1000),
  LogisticRegression(class_weight='balanced', max_iter=1000),
  LogisticRegression(class_weight='balanced', max_iter=1000),
  LogisticRegression(class_weight='balanced', max_iter=1000),
  LogisticRegression(class_weight='balanced', max_iter=1000),
  LogisticRegression(class_weight='balanced', max_iter=1000),
  LogisticRegression(class_weight='balanced', max_iter=1000),
  LogisticRegression(class_weight='balanced', max_iter=1000),
  LogisticRegression(class_weight='balanced', max_iter=1000),
  LogisticRegression(class_weight='balanced', max_iter=1000),
  LogisticRegression(class_weight='balanced', max_iter=1000),
  LogisticRegression(class_weight='balanced', max_iter=1000),
  LogisticRegression(class_weight='balanced', max_iter=1000)],
 [{'accuracy': 0.9855},
  {'accuracy': 0.9855},
  {'accuracy': 0.986},
  {'accuracy': 0.986},
  {'accuracy': 0.987},
  {'accuracy': 0.987},
  {'accuracy': 0.986},
  {'accuracy': 0.986},
  {'acc

# Negation Consistency Probe

In [27]:
train_dataset_neg_consistency = datasets.load_dataset("data_scripts/data_negation_consistency.py", add_sep=False)["train"]
test_dataset_neg_consistency = datasets.load_dataset("data_scripts/data_negation_consistency.py", add_sep=False)["test"]
dev_dataset_neg_consistency = datasets.load_dataset("data_scripts/data_negation_consistency.py", add_sep=False)["validation"]

Found cached dataset data_negation_consistency (/home2/shashwat.s/main_thread/negation_new/{'cache_dir': None, 'config_name': None, 'data_dir': None, 'data_files': None, 'hash': 'b61b143f8583c778d436a1730d12372367c64ff9c71127a6af2f230568607fe8', 'features': None, 'use_auth_token': None, 'base_path': 'data_scripts', 'add_sep': False}/data_negation_consistency/default/0.0.0)


  0%|          | 0/3 [00:00<?, ?it/s]

Found cached dataset data_negation_consistency (/home2/shashwat.s/main_thread/negation_new/{'cache_dir': None, 'config_name': None, 'data_dir': None, 'data_files': None, 'hash': 'b61b143f8583c778d436a1730d12372367c64ff9c71127a6af2f230568607fe8', 'features': None, 'use_auth_token': None, 'base_path': 'data_scripts', 'add_sep': False}/data_negation_consistency/default/0.0.0)


  0%|          | 0/3 [00:00<?, ?it/s]

Found cached dataset data_negation_consistency (/home2/shashwat.s/main_thread/negation_new/{'cache_dir': None, 'config_name': None, 'data_dir': None, 'data_files': None, 'hash': 'b61b143f8583c778d436a1730d12372367c64ff9c71127a6af2f230568607fe8', 'features': None, 'use_auth_token': None, 'base_path': 'data_scripts', 'add_sep': False}/data_negation_consistency/default/0.0.0)


  0%|          | 0/3 [00:00<?, ?it/s]

In [48]:
# checking for statistics
len(train_dataset_neg_consistency['label'])
total = 0
ones = 0

for label in train_dataset_neg_consistency['label']:
    total += 1
    ones += label

print(total)
print(ones)
print(ones/total)

23450
10066
0.4292537313432836


In [49]:
# checking for statistics
len(test_dataset_neg_consistency['label'])
total = 0
ones = 0

for label in test_dataset_neg_consistency['label']:
    total += 1
    ones += label

print(total)
print(ones)
print(ones/total)

5026
2160
0.4297652208515718


In [3]:
len(set(train_dataset_neg_consistency['subject']).intersection(set(test_dataset_neg_consistency['subject'])))

133

In [28]:
train_detect_neg_consistency, train_labels_det_neg_consistency = get_hidden_states_many_examples(model_wrapped, train_dataset_neg_consistency, n=8000, batch_size=100, query_column="subject")

In [29]:
test_detect_neg_consistency, test_labels_det_neg_consistency = get_hidden_states_many_examples(model_wrapped, test_dataset_neg_consistency, n=1000, batch_size=100, query_column="subject")

In [32]:
linear_classifiers, accuracies = run_experiment_across_layers(linear_probe_experiment, train_detect_neg_consistency, train_labels_det_neg_consistency, test_detect_neg_consistency, test_labels_det_neg_consistency)

/home2/shashwat.s/anaconda3/envs/new_research_env/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home2/shashwat.s/anaconda3/envs/new_research_env/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-le

In [33]:
mlps, accuracies = run_experiment_across_layers(mlp_probe_experiment, train_detect_neg_consistency, train_labels_det_neg_consistency, test_detect_neg_consistency, test_labels_det_neg_consistency)


In [34]:
reports = generate_classification_report_all_layers(test_detect_neg_consistency, test_labels_det_neg_consistency, mlps)

In [35]:
for report in reports[:3]:
    print(report)

              precision    recall  f1-score   support

           0       0.67      0.78      0.72       577
           1       0.61      0.49      0.54       423

    accuracy                           0.65      1000
   macro avg       0.64      0.63      0.63      1000
weighted avg       0.65      0.65      0.65      1000

              precision    recall  f1-score   support

           0       0.68      0.72      0.70       577
           1       0.58      0.54      0.56       423

    accuracy                           0.64      1000
   macro avg       0.63      0.63      0.63      1000
weighted avg       0.64      0.64      0.64      1000

              precision    recall  f1-score   support

           0       0.70      0.68      0.69       577
           1       0.58      0.61      0.59       423

    accuracy                           0.65      1000
   macro avg       0.64      0.64      0.64      1000
weighted avg       0.65      0.65      0.65      1000



In [36]:
for report in reports[3:6]:
    print(report)

              precision    recall  f1-score   support

           0       0.70      0.71      0.71       577
           1       0.60      0.59      0.59       423

    accuracy                           0.66      1000
   macro avg       0.65      0.65      0.65      1000
weighted avg       0.66      0.66      0.66      1000

              precision    recall  f1-score   support

           0       0.68      0.73      0.71       577
           1       0.59      0.53      0.56       423

    accuracy                           0.65      1000
   macro avg       0.64      0.63      0.63      1000
weighted avg       0.64      0.65      0.64      1000

              precision    recall  f1-score   support

           0       0.69      0.75      0.72       577
           1       0.61      0.53      0.57       423

    accuracy                           0.66      1000
   macro avg       0.65      0.64      0.64      1000
weighted avg       0.66      0.66      0.66      1000



In [37]:
for report in reports[6:9]:
    print(report)

              precision    recall  f1-score   support

           0       0.68      0.75      0.71       577
           1       0.60      0.51      0.55       423

    accuracy                           0.65      1000
   macro avg       0.64      0.63      0.63      1000
weighted avg       0.64      0.65      0.64      1000

              precision    recall  f1-score   support

           0       0.68      0.76      0.72       577
           1       0.61      0.51      0.55       423

    accuracy                           0.66      1000
   macro avg       0.64      0.64      0.64      1000
weighted avg       0.65      0.66      0.65      1000

              precision    recall  f1-score   support

           0       0.69      0.76      0.72       577
           1       0.62      0.53      0.58       423

    accuracy                           0.67      1000
   macro avg       0.66      0.65      0.65      1000
weighted avg       0.66      0.67      0.66      1000



In [38]:
for report in reports[9:12]:
    print(report)

              precision    recall  f1-score   support

           0       0.72      0.70      0.71       577
           1       0.61      0.63      0.62       423

    accuracy                           0.67      1000
   macro avg       0.66      0.66      0.66      1000
weighted avg       0.67      0.67      0.67      1000

              precision    recall  f1-score   support

           0       0.71      0.75      0.73       577
           1       0.63      0.58      0.60       423

    accuracy                           0.68      1000
   macro avg       0.67      0.67      0.67      1000
weighted avg       0.68      0.68      0.68      1000

              precision    recall  f1-score   support

           0       0.70      0.71      0.70       577
           1       0.59      0.58      0.59       423

    accuracy                           0.65      1000
   macro avg       0.64      0.64      0.64      1000
weighted avg       0.65      0.65      0.65      1000



In [39]:
for report in reports[12:]:
    print(report)

              precision    recall  f1-score   support

           0       0.69      0.74      0.72       577
           1       0.61      0.56      0.58       423

    accuracy                           0.66      1000
   macro avg       0.65      0.65      0.65      1000
weighted avg       0.66      0.66      0.66      1000



In [46]:
test_dataset_neg_consistency["masked_non_negated"][:5000]

['Piero Gobetti used to communicate in <mask>.',
 'The original language of Thulladha Manamum Thullum is <mask>.',
 'Fiat Siena is produced by <mask>.',
 'ferric phosphate consists of <mask>.',
 'Carlos Fuentes used to communicate in <mask>.',
 'National Assembly of Hungary is a legal term in <mask>.',
 'Dennistoun Glacier is located in <mask>.',
 'Sasha Krasny died in the city of <mask>.',
 'Pakistan shares border with <mask>.',
 'Dream with Me was written in <mask>.',
 'Hamilton Central railway station is named after <mask>.',
 'West Royalty, Prince Edward Island is located in <mask>.',
 'Dyfed Archaeological Trust works in the field of <mask>.',
 'Florian Henckel von Donnersmarck used to communicate in <mask>.',
 'Edmund Hobhouse was born in the city of <mask>.',
 'The official language of Idaho is <mask>.',
 'Sir Thomas Buxton, 1st Baronet used to work in <mask>.',
 'Pro-feminism is part of <mask>.',
 'Yotam Halperin used to communicate in <mask>.',
 'Newfoundland expedition is loc

In [35]:
train_dataset_neg_detect["content"][:5000]


['Library of Alexandria',
 'Bruno Racine',
 'Salimuzzaman Siddiqui',
 "Kay O'Brien",
 'Antim Peak',
 'Adobe Media Player',
 'Cauchy stress tensor',
 'Raymond Queneau',
 'Sweden',
 'Chrysler K platform',
 'El Siglo Futuro',
 'Johan Gustaf Sandberg',
 'Fiat Uno',
 'Louisiana Voodoo',
 'Jean-Baptiste Henri Lacordaire',
 'aerial photography',
 'Chrysalis',
 'Charles Augustin Sainte-Beuve',
 'Super Monaco GP',
 'Abraham Blooteling',
 'Edward Bulwer-Lytton',
 "Pourvu qu'elles soient douces",
 'The Bill Cosby Show',
 'Pauline Mills McGibbon',
 'Maryland General Assembly',
 'Average Joe',
 'Singapore International Film Festival',
 'Munich Re',
 'Delhi Technological University',
 'Ukrainian Railways',
 'President of Vietnam',
 'BBC Persian Television',
 'Mishor Rohoshyo',
 'Super Monaco GP',
 'Western Canada Concept',
 'Sardinian Judicati',
 'Sony Crackle',
 'Christian Wilhelm Blomstrand',
 'Florence Regional Airport',
 'Little Child',
 'nickel silver',
 'Jan van Krimpen',
 'Uppslagsverket Finl

## Negation consistency given entire sentence

In [40]:
train_detect_neg_consistency_ful, train_labels_det_neg_consistency = get_hidden_states_many_examples(model_wrapped, train_dataset_neg_consistency, n=23000, batch_size=100, query_column="masked_non_negated")


In [41]:
test_detect_neg_consistency_ful, test_labels_det_neg_consistency = get_hidden_states_many_examples(model_wrapped, test_dataset_neg_consistency, n=5000, batch_size=100, query_column="masked_non_negated")


In [42]:
mlps, accuracies = run_experiment_across_layers(mlp_probe_experiment, train_detect_neg_consistency_ful, train_labels_det_neg_consistency, test_detect_neg_consistency_ful, test_labels_det_neg_consistency)



In [53]:
reports = generate_classification_report_all_layers(test_detect_neg_consistency_ful, test_labels_det_neg_consistency, mlps)


In [55]:
for report in reports[0:3]:
    print(report)

              precision    recall  f1-score   support

           0       0.79      0.80      0.79       954
           1       0.73      0.71      0.72       721

    accuracy                           0.76      1675
   macro avg       0.76      0.75      0.76      1675
weighted avg       0.76      0.76      0.76      1675

              precision    recall  f1-score   support

           0       0.77      0.83      0.80       954
           1       0.75      0.68      0.71       721

    accuracy                           0.77      1675
   macro avg       0.76      0.76      0.76      1675
weighted avg       0.76      0.77      0.76      1675

              precision    recall  f1-score   support

           0       0.79      0.79      0.79       954
           1       0.73      0.72      0.72       721

    accuracy                           0.76      1675
   macro avg       0.76      0.76      0.76      1675
weighted avg       0.76      0.76      0.76      1675



In [54]:
for report in reports[6:9]:
    print(report)

              precision    recall  f1-score   support

           0       0.82      0.83      0.82       954
           1       0.77      0.76      0.76       721

    accuracy                           0.80      1675
   macro avg       0.79      0.79      0.79      1675
weighted avg       0.80      0.80      0.80      1675

              precision    recall  f1-score   support

           0       0.82      0.80      0.81       954
           1       0.74      0.76      0.75       721

    accuracy                           0.78      1675
   macro avg       0.78      0.78      0.78      1675
weighted avg       0.78      0.78      0.78      1675

              precision    recall  f1-score   support

           0       0.81      0.82      0.82       954
           1       0.76      0.74      0.75       721

    accuracy                           0.79      1675
   macro avg       0.78      0.78      0.78      1675
weighted avg       0.79      0.79      0.79      1675



In [56]:
for report in reports[9:]:
    print(report)

              precision    recall  f1-score   support

           0       0.82      0.83      0.83       954
           1       0.77      0.76      0.77       721

    accuracy                           0.80      1675
   macro avg       0.80      0.80      0.80      1675
weighted avg       0.80      0.80      0.80      1675

              precision    recall  f1-score   support

           0       0.80      0.85      0.83       954
           1       0.78      0.73      0.75       721

    accuracy                           0.80      1675
   macro avg       0.79      0.79      0.79      1675
weighted avg       0.80      0.80      0.80      1675

              precision    recall  f1-score   support

           0       0.81      0.84      0.82       954
           1       0.77      0.73      0.75       721

    accuracy                           0.79      1675
   macro avg       0.79      0.79      0.79      1675
weighted avg       0.79      0.79      0.79      1675

              preci